In [1]:
from tempo.serve.metadata import ModelFramework, KubernetesOptions
from tempo.serve.model import Model
from tempo.seldon.deploy import SeldonDeployRuntime
import os
import pprint
import numpy as np

rt = SeldonDeployRuntime(host="http://34.105.136.157/seldon-deploy/api/v1alpha1",
                        user="admin@kubeflow.org",
                        password= "12341234",
                        k8s_options=KubernetesOptions(namespace="seldon"))

sklearn_model = Model(
    name="test-iris-sklearn",
    runtime=rt,
    platform=ModelFramework.SKLearn,
    uri="gs://seldon-models/sklearn/iris",
    local_folder=os.getcwd() + "/sklearn")


In [2]:
yaml = sklearn_model.to_k8s_yaml()
print (eval(pprint.pformat(yaml)))

/home/clive/anaconda3/envs/mlops/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: test-iris-sklearn
  namespace: seldon
spec:
  predictors:
  - graph:
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1



In [3]:
sklearn_model.deploy()

In [4]:
sklearn_model.get_endpoint()

'http://34.105.136.157/seldon/seldon/test-iris-sklearn/api/v1.0/predictions'

In [6]:
sklearn_model.wait_ready()

True

In [12]:
sklearn_model(np.array([[4.9, 3.1, 1.5, 0.2]]))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
sklearn_model.undeploy()